## Project- AirFares
--- -------------------

### A. Introduction
--- -------------------

This project studies the german domestic airfares during the period 25-10-2019 to 24-04-2020 between major airports in Germany. The aim of the project is to apply the techniques of machine learning on the dataset and understand the trends in pricing with respect to the various features such the booking date, departure & arrival cities, departure time etc. 
#### B. Details of dataset:
-- -------------------
1. Source: [https://www.kaggle.com/datasets/darjand/domestic-german-air-fares](https://www.kaggle.com/datasets/darjand/domestic-german-air-fares)
2. Generation mode: web scraping
3. Time period considered: 25-10-2019 to 24-04-2020 (6 months).
4. Total entries: 63,000
5. Features:
    * departure_city: The city from which the flight departs.
    * arrival_city: The city to which the flight arrives.
    * scrape_date: The date when flight price information was retrieved.
    * departure_date: The departure date of the flight (25-10-2019 to 24-04-2020).
    * departure_date_distance: How far in advance (e.g., "1 week") the flight was booked.
    * departure_time: The departure time of the flight.
    * arrival_time: The arrival time of the flight.
    * airline: The airline that operates the flight.
    * stops: The number of layovers or stops during the flight.
    * price (€): The price of the flight ticket in Euros.

#### C. Imports:
-- ----------

In [15]:
#imports
import numpy as np
import pandas as pd
import sklearn.model_selection as ms
import sklearn.feature_extraction.text as text
import sklearn.naive_bayes as nb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from datetime import datetime

pd.set_option('display.max_colwidth', 50)


#### D. Load the dataset:
-- ------------------

In [16]:
#Load the data set
df = pd.read_csv('./data/German Air Fares.csv');

#### E. Data Understanding:
-- ----------------------

##### E.1 Basic statistics:

In [17]:
#Basic statistics
df.shape;                                           #--> (62626, 10)
df.info();df.isna().sum();                          #--> (No null objects)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62626 entries, 0 to 62625
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   departure_city           62626 non-null  object
 1   arrival_city             62626 non-null  object
 2   scrape_date              62626 non-null  object
 3   departure_date           62626 non-null  object
 4   departure_date_distance  62626 non-null  object
 5   departure_time           62626 non-null  object
 6   arrival_time             62626 non-null  object
 7   airline                  62626 non-null  object
 8   stops                    62626 non-null  object
 9   price (€)                62626 non-null  object
dtypes: object(10)
memory usage: 4.8+ MB


##### E.2 Understanding departure and arrival cities
-- ----------------------

In [18]:
df.departure_city.unique();
# a. Extract departure city name, discard shortform (e.g. DUS Düsseldorf -> Düsseldorf)
df['departure_city'] = df['departure_city'].astype(str).apply(lambda x: x.split()[1] if ' ' in x else x)

df.arrival_city.unique();
# b. Extract arrival city name, discard shortform (e.g. DUS Düsseldorf -> Düsseldorf)
df['arrival_city'] = df['arrival_city'].astype(str).apply(lambda x: x.split()[1] if ' ' in x else x)

##### E.2 Conclusion: 
* **ToDos:**
    * *A histogram showing distribution of departure and arrival cities would be interesting.*</font>
    * <font color='green'>**Additional Feature:** 
        * The cities needs to be converted into real numbers, based on some metric (e.g. `'ranking of the 'airports') in order to be able to be used in linear regression? 
    Alternatively, can we use the histogram distribution to assign a ranking to the cities? </font>
    * <font color='red'>**Feedback from professor:**
        * Do not use histogram, because the data distribution might not be representative of real world.</font>
    * **Assigned to:** Yassine

##### E.3 Understanding departure and scrape dates
-- ----------------------

In [19]:
# a. Departure dates: Find the departure date frequencies: How spread are they?
dep_dates = pd.to_datetime(df['departure_date'], format='%d.%m.%Y');
dep_dates = sorted(dep_dates.unique()); 
dep_dates = pd.Series(dep_dates);
dep_dates.shape;                                    
dep_dates.diff(periods=1).unique();        

# b. Scrape dates: Find the scrape date frequencies: How spread are they?
scrape_dates = pd.to_datetime(df['scrape_date'], format='%d.%m.%Y');
scrape_dates = sorted(scrape_dates.unique()); 
scrape_dates = pd.Series(scrape_dates);
scrape_dates.shape;                                    
scrape_dates.diff(periods=1).unique();

#--> Additionally, it was verified that there are no illegal date values.        

#--------------
#Feature: Departure weekdays
df.departure_date = pd.to_datetime(df.departure_date, dayfirst=True);
df['departure_weekday'] = df.departure_date.dt.weekday

##### E3: Conclusion: 
* **Departure dates:**
    * 42 unique departure dates
    * departure date frequencies are not unique -> ['1 days', '11 days', '5 days', '44 days', '85 days']
* **Scrape dates:**
    * scrape date frequencies are only from 18-24.10.2019
    * scrape date frequencies are unique -> 1 day
* **ToDos:**
    * <font color='green'>**Additional Feature:** 
        * <font color='red'>**Feedback from professor:**
            * Do not use the dates as such as features; rather convert the departure date to a weekday (e.g. Sunday, Monday etc.) and use it as feature.</font> -> DONE
        * A new feature 'days until a holiday' from a departure date to the nearest holiday could be added as an additional feature
        </font>
    * **Assigned to:**

In [46]:
df.head(20)


,departure_city,arrival_city,scrape_date,departure_date,departure_date_distance,departure_time,arrival_time,airline,stops,price,departure_weekday
0,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,06:30:00,07:45:00,Eurowings,0,74,4
1,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,06:40:00,07:55:00,EasyJet,0,75,4
2,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,06:45:00,08:00:00,EasyJet,0,80,4
3,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,06:15:00,07:30:00,Eurowings,0,84,4
4,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,06:55:00,08:10:00,Eurowings,0,84,4
5,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,08:00:00,09:10:00,Eurowings,0,104,4
6,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,08:05:00,09:20:00,EasyJet,0,116,4
7,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,21:25:00,22:40:00,EasyJet,0,131,4
8,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,20:40:00,21:50:00,Eurowings,0,135,4
9,Berlin-Tegel,Düsseldorf,18.10.2019,2019-10-25,1 week,08:20:00,09:35:00,EasyJet,0,150,4


##### E.4  Understanding departure-date distance
-- ----------------------------------------

In [20]:
df.departure_date_distance.value_counts();           
#--> 
# 6 months    12672
# 6 weeks     11222
# 1 month     10092
# 1 week       9949
# 3 month      9748
# 2 weeks      7850
# 2 week       1093


##### E.4 Conclusion: 
* **Departure-date distance:**
    * scrape_date can probably be omitted, redundant feature?
    * <font color='red'>**ToDos:**
        * Convert the weeks and months in days, in order to have a uniform format for regression tasks</font>

##### E.5 Understanding departure and arrival times
-- ---------------------------------------------

In [21]:
#To-Do:
#Time formats are not unique:
#--> am/pm and 24 hr formats mixed ->DONE
#--> no consistent display schema ->DONE
#--> convert time into number equivalents, in order to be usable in e.g., regression?
count = 0
for i in df['departure_time']:
    if(i.find('Uhr') >= 0):
        df['departure_time'][count] = datetime.strptime(i, '%H:%M Uhr').time()
    else:
        df['departure_time'][count] = datetime.strptime(i, '%I:%M%p').time()
    count += 1
    
count = 0
for i in df['arrival_time']:
    if(i.find('Uhr') >= 0):
        df['arrival_time'][count] = datetime.strptime(i, '%H:%M Uhr').time()
    else:
        df['arrival_time'][count] = datetime.strptime(i, '%I:%M%p').time()
    count += 1

##### E.5 Conclusion: 
* **ToDos:**
    * <font color='red'>**Feedback from professor:**
        * The time data needs to be converted into real numbers. Convert times into hours only (e.g. 12:30 and 12:45 --> just 12) for easier computation. The values would be then in range 0 -> 24
        * Use departure times only as feature, no need of arrival time, to keep it simple</font>
    * *A histogram showing distribution of departure and arrival time intervals would be interesting.*</font>
    * <font color='green'>**Additional Feature:**
        * The travel time (arrival_time - departure_time) could be a new feature, in addition to the departure and arrival times itself</font>
    * **Assigned to:** Dominik
    

##### E.6 Understanding Airlines
-- ---------------------------------------------

In [22]:
pd.options.mode.chained_assignment = None
mul = []
count = 0
for i in df['airline']:
    if(i == 'Mehrere Fluglinien' or i == 'Multiple Airlines'):
        mul.append(count)
    df['airline'][count] = i.replace('easyJet', 'EasyJet') # Rename 'easyjet' to 'Easyjet'
    count += 1
df.drop(mul, axis=0, inplace=True)  # Drop rows with 'Mehrere Fluglinien' and 'Multiple Airlines'
df.reset_index(drop=True, inplace=True)
df.airline.value_counts();
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62404 entries, 0 to 62403
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   departure_city           62404 non-null  object
 1   arrival_city             62404 non-null  object
 2   scrape_date              62404 non-null  object
 3   departure_date           62404 non-null  object
 4   departure_date_distance  62404 non-null  object
 5   departure_time           62404 non-null  object
 6   arrival_time             62404 non-null  object
 7   airline                  62404 non-null  object
 8   stops                    62404 non-null  object
 9   price (€)                62404 non-null  object
dtypes: object(10)
memory usage: 4.8+ MB


##### E.6 Conclusion:
* **Airlines:**
    * There were 69 'Mehrere Fluglinien' and 153 'Multiple Airlines', these entries were removed.
    * <font color='green'>**Additional Feature:**
        * Similar to E2, the airlines needs to be converted into real numbers, based on some metric in order to be able to be used in linear regression? Can we use the histogram distribution to assign a ranking to the airlines?</font>
    * <font color='red'>**Feedback from professor:**
        * Do not use histogram, because the data distribution might not be representative of real world.</font>
    * **Assigned to:** Yassine
    

##### E.7 Understanding stops
---------------------------------------------

In [23]:
#--> Rename stops to numerical data ->DONE
count = 0
for i in df['stops']:
    if(i.find('d') >= 0):
        df['stops'][count] = 0
    elif(i.find('2') >= 0):
        df['stops'][count] = 2
    else:
        df['stops'][count] = 1
    count += 1
df['stops'] = df['stops'].astype(int);
df.stops.value_counts();

##### E.7 Conclusion:
* Stops are converted into numerical values
    

##### E8. Understanding prices
--------------------------


In [24]:
# a. Rename 'price (€)' to 'price'
df = df.rename(columns={'price (€)': 'price'});

# b. format into int properly
count = 0
for i in df['price']:
    if(i.find(',') >= 0):
        df['price'][count] = i.replace(',', '') #-> e.g. 1,047 to 1047
    count += 1
df['price'] = df['price'].astype(int)

##### E.8 Conclusion:
* prices are, of course, the target value